In [1]:
import pandas as pd
import sklearn
import numpy as np

In [2]:
rate = pd.read_csv("data.csv")

In [3]:
rate.head()

,ReverseRepoRate,BankRate,BaseRate,SavingsDepositRate,Inflation,RepoRate
0,4.9,5.4,8.45,3.25,6.62,5.15
1,4.9,5.4,8.45,3.25,6.62,5.15
2,4.9,5.4,8.45,3.25,3.73,5.15
3,4.9,5.4,8.45,3.25,3.73,5.15
4,4.9,5.4,8.45,3.25,3.73,5.15


In [4]:
rate.shape

(33048, 6)

In [5]:
rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33048 entries, 0 to 33047
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ReverseRepoRate     33048 non-null  float64
 1   BankRate            33048 non-null  float64
 2   BaseRate            33048 non-null  float64
 3   SavingsDepositRate  33048 non-null  float64
 4   Inflation           33048 non-null  float64
 5   RepoRate            33048 non-null  float64
dtypes: float64(6)
memory usage: 1.5 MB


## Linear Regression Model

In [6]:
X= rate.drop(columns='RepoRate')
y= rate['RepoRate']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2)

In [8]:
from sklearn.linear_model import LinearRegression
##from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [9]:
ohe = OneHotEncoder()
ohe.fit(X[['ReverseRepoRate','BankRate', 'BaseRate','SavingsDepositRate','Inflation']])

OneHotEncoder()

In [10]:
ohe.categories_

[array([4.9 , 5.15, 5.5 , 5.75, 6.  , 6.25, 6.5 , 6.75, 7.  , 7.25, 7.5 ]),
 array([ 5.4 ,  5.65,  6.  ,  6.25,  6.5 ,  6.75,  7.  ,  7.75,  8.25,
         8.5 ,  8.75,  9.  ,  9.5 , 10.25]),
 array([ 8.25,  8.45,  8.5 ,  8.95,  9.  ,  9.1 ,  9.25,  9.3 ,  9.5 ,
         9.6 ,  9.7 ,  9.75,  9.8 , 10.  ]),
 array([0.  , 3.25, 3.5 , 4.  ]),
 array([ 3.33,  3.73,  3.94,  4.91,  4.95,  6.62,  6.67,  8.91,  9.48,
        10.02])]

In [11]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['ReverseRepoRate','BankRate', 'BaseRate', 'SavingsDepositRate','Inflation']),remainder= 'passthrough')

In [12]:
lr=LinearRegression()

In [13]:
pipe = make_pipeline(column_trans,lr)

In [14]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array([4.9 , 5.15, 5.5 , 5.75, 6.  , 6.25, 6.5 , 6.75, 7.  , 7.25, 7.5 ]),
                                                                            array([ 5.4 ,  5.65,  6.  ,  6.25,  6.5 ,  6.75,  7.  ,  7.75,  8.25,
        8.5 ,  8.75,  9.  ,  9.5 , 10.25]),
                                                                            array([ 8.25,  8.45,  8.5 ,  8.95,  9.  ,  9.1 ,  9.25,  9.3 ,  9.5 ,
        9.6 ,  9.7 ,  9.75,  9.8 , 10.  ]),
                                                                            array([0.  , 3.25, 3.5 , 4.  ]),
                                                                            array([ 3.33,  3.73,  3.94,  4.91,  4.95,  6.62,  6.67,  8.91,  9.48,
       10.02])]),
                                                  ['ReverseRepoRate',
                                                   'BankRate', 'BaseRate',
                                                   'SavingsDepositRate',
                                                   'Inflation'])])),
                ('linearregression', LinearRegression())])

In [15]:
y_pred= pipe.predict(X_test)

In [16]:
r2_score(y_test,y_pred)

0.9996472527782697

In [17]:
scores=[]
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [18]:
np.argmax(scores)

7

In [19]:
scores[np.argmax(scores)]

0.9996769793845598

In [20]:
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    r2_score(y_test,y_pred)
    

0.9996769793845598

In [21]:
import pickle

In [22]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [23]:
pipe.predict(pd.DataFrame([[4.9, 5.4, 8.45, 3.25,6.62]], columns=	['ReverseRepoRate',	'BankRate',	'BaseRate',	'SavingsDepositRate', 'Inflation']))

array([5.14999468])